In [1]:
### importing all the libraries

from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import requests
import csv
import time
import pickle

# YOUTUBE SCRAPING

In [2]:
### A helper function to extract all the links for a particular topic

def link(driver,links,file):
    
    time.sleep(15)
    user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
    
    for i in user_data:
        if str(i.get_attribute('href')) not in links and str(i.get_attribute('href'))!='None':
               links.append(str(i.get_attribute('href')))
        else:
               continue
    print(len(links))
    
    if (len(links))>=170:
        driver.close() ##close the browser window
        a = open(file, 'w')
        a.write(" ".join(links))
        a.close()
        return links
        
    else:
        driver.execute_script("window.scrollBy(0,4300)","") ##scrolls the window of browser vertically
        link(driver,links,file)


In [3]:
### A function to extract all the links for a particular topic

def get_links(data,links,file):
    
    driver = webdriver.Chrome(executable_path = 'G:\youtube scraping\chromedriver_win32\chromedriver.exe')
    ## It open a chrome browser
    
    driver.get("https://www.youtube.com/results?search_query=" + str(data))
    ## browser is opened with this link
    
    link(driver,links,file)
    

In [4]:
### A function to get all the details of a particular topic

def get_df(link,data):
    
        driver = webdriver.Chrome(executable_path = 'G:\youtube scraping\chromedriver_win32\chromedriver.exe')
        driver.get(link)
        ## It open a chrome browser with a particular link
            
        wait = WebDriverWait(driver,45)
        
        v_id = link.strip('https://www.youtube.com/watch?v=') ## extracts video_id 
           
        v_title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text
        ## extracts video_title
                
        v_description =  wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div#description yt-formatted-string"))).text
        ## extract video_description
        
        v_category=data
            
        driver.close() ## closes the browser window
        row=  [v_id, v_title, v_description, v_category]  
        
        return row
    
            
    
     
    
    

### Extracting links for every topic

In [31]:
data_t='Travel Blogs'
links_tb = []
links_Travel_Blogs=get_links(data_t,links_tb,'links_Travel_Blogs.txt')

23
43
56
69
83
94
106
116
129
143
155
170


In [32]:
data_st='Science and Technology'
links_st = []
links_Science_and_Technology=get_links(data_st,links_st,'links_Science_and_Technology.txt')

19
34
51
65
77
92
103
121
137
150
165
183


In [33]:
data_f='Food'
links_f = []
links_Food=get_links(data_f,links_f,'links_Food.txt')

19
38
57
75
92
109
128
145
161
179


In [34]:
data_m='Manufacturing'
links_m = []
links_Manufacturing=get_links(data_m,links_m,'links_Manufacturing.txt')

20
36
56
72
91
108
125
140
154
169
186


In [35]:
data_h='History'
links_h = []
links_History=get_links(data_h,links_h,'links_History.txt')

21
41
61
80
98
118
136
155
172


In [36]:
data_am='Art_and_Music'
links_am = []
links_Art_and_Music=get_links(data_am,links_am,'links_Art_and_Music.txt')

20
35
48
62
78
93
105
121
139
156
172


## Extract Details of all the topics

### Travel_Blogs

In [95]:
list_Travel_Blogs=[]
for i in range(170):
    list_Travel_Blogs.append(get_df(links_Travel_Blogs[i],'Travel_Blogs'))
    

### Science_and_Technology

In [96]:
list_Science_and_Technology=[]
for i in range(70):
    list_Science_and_Technology.append(get_df(links_Science_and_Technology[i],'Science_and_Technology'))

### Manufacturing

In [97]:
list_Manufacturing=[]
for i in range(170):
    list_Manufacturing.append(get_df(links_Manufacturing[i],'Manufacturing'))

### Food

In [98]:
list_Food=[]
for i in range(171):
    list_Food.append(get_df(links_Food[i],'Food'))

### History

In [23]:
list_History=[]
for i in range(170):
    list_History.append(get_df(links_History[i],'History'))


### Art_and_Music

In [99]:
list_Art_and_Music=[]
for i in range(170):
    list_Art_and_Music.append(get_df(links_Art_and_Music[i],'Art_and_Music'))

In [14]:
### Adding all the data in a list 
l=list_Travel_Blogs+list_Art_and_Music+list_History+list_Science_and_Technology+list_Food+list_Manufacturing

# CLEANING THE DATA

In [15]:
### import all the libraries

import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [16]:
extra_words=stopwords.words('english')+list(string.punctuation)
lemm = WordNetLemmatizer()

In [17]:
###  finding pos

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):##adjective
        return 'a'
    elif treebank_tag.startswith('V'):##verb
        return 'v'
    elif treebank_tag.startswith('N'):##noun
        return 'n'
    elif treebank_tag.startswith('R'):##adverb
        return 'r'
    else:
        return ''

In [18]:
### cleaning the document

def clean_doc(doc):
    tagged = nltk.pos_tag(nltk.word_tokenize(doc))
    #print(tagged)
    new_data=[]
    for i,j in tagged:
        if i.lower() not in extra_words and i.isalpha()==True and len(i)>2:
            try:
                a=get_wordnet_pos(j)
                new_word=lemm.lemmatize(i.lower(),pos=a)
                new_data.append(new_word.lower())
            except:
                new_data.append(i.lower())
            
            
    return ' '.join(new_data),new_data
            
    


In [19]:
### Cleaning the title
for i in l:
    i[1]=clean_doc(i[1])[0]

In [20]:
### Cleaning the description
all_words=[]
doc=[]
for i in l:
    i[2]=clean_doc(i[2])[0]
    all_words+=clean_doc(i[2])[1]
    doc.append((clean_doc(i[2])[0],i[3]))

In [67]:
doc[0]

('http stay current recent uploads update channel youtube', 'Art_and_Music')

### Creating DataFrame

In [24]:
data=pd.DataFrame(l,columns = ['Video id',' Title',' Description',' Category' ])

In [32]:
data=data.drop_duplicates()

In [33]:
data.head()

,Video id,Title,Description,Category
0,JLZlCZ0,travelling north east india arunachal journey ...,journey arunachal north east india begin train...,Travel_Blogs
1,i9E_Blai8vk,welcome bali travel vlog priscilla lee,chance fly bali whole family thanksgiving firs...,Travel_Blogs
2,n79Rv0F48,backpack meghalaya northeast india trip sohra ...,video explore north east india start guwahati ...,Travel_Blogs
3,i5F7Xh9CO8U,exploring varanasi benaras travel vlog,spent incredible week explore beautiful city v...,Travel_Blogs
4,RcmrbNRK-jY,day trip around world travel film,wife travel country day film story incredible ...,Travel_Blogs


In [68]:
data.to_csv('youtubeScrap.csv',index=False)

In [72]:
data_ys=pd.read_csv('youtubeScrap.csv')

In [73]:
data_ys.columns

Index(['Video id', ' Title', ' Description', ' Category'], dtype='object')

In [75]:
data_ys.loc[0,' Description']

'journey arunachal north east india begin train journey guwahati murkongselek head pasighat travel companion get start explore tiny glimpe arunachal far market bridge adventure get well next video subscribe channel hit bell icon stay notified follow social medium instagram facebook twitter website check gear kit music epidemic music hear epidemic sound sign use link get month free subscription trip unknown arunachal pradesh november november organise chalohoppo travel company specialise craft authentic local experience northeast india experience raw nothing like anything experience check instagram facebook website shot traveller edit premiere pro'

In [76]:
text=data_ys[' Description']
target=data_ys[' Category']

In [77]:
### import countvectorizer

from sklearn.feature_extraction.text import CountVectorizer

In [78]:
### split data into train and test

from sklearn.model_selection import train_test_split
trainX,testX,trainY,testY=train_test_split(text,target,random_state=1,test_size=0.2)

In [81]:
count_vex=CountVectorizer(max_features=3000, analyzer='word', encoding = "ISO-8859-1")
train_feature=count_vex.fit_transform(trainX.values.astype('U'),trainY)
test_feature=count_vex.transform(testX.values.astype('U'))

In [82]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(train_feature,trainY)
y_score=lr.score(test_feature,testY)

C:\Users\Varsha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Varsha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [83]:
y_score

0.8275862068965517